In [31]:
import pandas as pd
import numpy as np

In [32]:
import os
print(os.listdir("/home/sunilvikram/data"))

['mnt', 'sample_submission.csv.zip', 'test_supplement.csv.zip', '.ipynb_checkpoints', 'submission.csv', 'train_sample.csv.zip', 'Dalon_notebook.ipynb', 'xgboost_8c.pkl', 'sample_submission.csv', 'test.csv', 'train.csv.zip', 'test.csv.zip', 'BuildModel.ipynb']


In [33]:
usefulcolumns=['ip','app','device','os', 'channel', 'click_time', 'is_attributed']
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        }

In [4]:
traindata= pd.read_csv('mnt/ssd/kaggle-talkingdata2/competition_files/train.csv',parse_dates=['click_time'],dtype=dtypes,usecols=usefulcolumns)

In [5]:
traindata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184903890 entries, 0 to 184903889
Data columns (total 7 columns):
ip               uint32
app              uint16
device           uint16
os               uint16
channel          uint16
click_time       datetime64[ns]
is_attributed    uint8
dtypes: datetime64[ns](1), uint16(4), uint32(1), uint8(1)
memory usage: 3.6 GB


In [6]:
AttributedDF= traindata[traindata.is_attributed==1]

In [7]:
NotAttributed= traindata[traindata.is_attributed==0]

In [8]:
NotAttributed.shape

(184447044, 7)

In [9]:
NonAttrCount=AttributedDF.shape[0]*10 -AttributedDF.shape[0]

In [10]:
NonAttrCount= NonAttrCount *10

In [11]:
#lines=NotAttributed.shape[0]
#howmanytoskip = lines-NonAttrCount

#skiplines = np.random.choice(np.arange(1, lines), size=NonAttrCount, replace=False)

#sort the list
#skiplines=np.sort(skiplines)

In [11]:
NotAttributed=NotAttributed.sample(n=NonAttrCount)

In [12]:
NotAttributed.shape

(41116140, 7)

In [13]:
newdf = [AttributedDF,NotAttributed]

In [14]:
newtraindata= pd.concat(newdf)

In [15]:
newtraindata.shape

(41572986, 7)

In [16]:
del traindata


In [17]:
import gc
gc.collect()

32

In [18]:
del NotAttributed
del AttributedDF
gc.collect()

0

In [19]:
def timeFeature(df):
    df['dayofclick'] = df['click_time'].dt.day.astype('uint8')
    df['hourofclick'] = df['click_time'].dt.hour.astype('uint8')
    df['minofclick'] = df['click_time'].dt.minute.astype('uint8')
    df['secofclick'] = df['click_time'].dt.second.astype('uint8')
    return df

In [20]:
newtraindata=timeFeature(newtraindata)
newtraindata.head()

,ip,app,device,os,channel,click_time,is_attributed,dayofclick,hourofclick,minofclick,secofclick
103,204158,35,1,13,21,2017-11-06 15:41:07,1,6,15,41,7
1504,29692,9,1,22,215,2017-11-06 16:00:02,1,6,16,0,2
1798,64516,35,1,13,21,2017-11-06 16:00:02,1,6,16,0,2
2102,172429,35,1,46,274,2017-11-06 16:00:03,1,6,16,0,3
3056,199085,35,1,13,274,2017-11-06 16:00:04,1,6,16,0,4


In [21]:
import gc
gc.collect()

121

In [36]:
# Define all the groupby transformations
def groupByFeatures(df):
    GROUPBY_AGGREGATIONS = [
    
    # V1 - GroupBy Features #
    #########################    
    # Variance in day, for ip-app-channel
    {'groupby': ['ip','app','channel'], 'select': 'dayofclick', 'agg': 'var'},
    # Variance in hour, for ip-app-os
    {'groupby': ['ip','app','os'], 'select': 'hourofclick', 'agg': 'var'},
    # Variance in hour, for ip-day-channel
    {'groupby': ['ip','dayofclick','channel'], 'select': 'hourofclick', 'agg': 'var'},
    # Count, for ip-day-hour
    {'groupby': ['ip','dayofclick','hourofclick'], 'select': 'channel', 'agg': 'count'},
    # Count, for ip-app
    {'groupby': ['ip', 'app'], 'select': 'channel', 'agg': 'count'},        
    # Count, for ip-app-os
    {'groupby': ['ip', 'app', 'os'], 'select': 'channel', 'agg': 'count'},
    # Count, for ip-app-day-hour
    {'groupby': ['ip','app','dayofclick','hourofclick'], 'select': 'channel', 'agg': 'count'},
    # Mean hour, for ip-app-channel
    {'groupby': ['ip','app','channel'], 'select': 'hourofclick', 'agg': 'mean'}, 
    
    # V2 - GroupBy Features #
    #########################
    # Average clicks on app by distinct users; is it an app they return to?
    {'groupby': ['app'], 
     'select': 'ip', 
     'agg': lambda x: float(len(x)) / len(x.unique()), 
     'agg_name': 'AvgViewPerDistinct'
    },
    # How popular is the app or channel?
    {'groupby': ['app'], 'select': 'channel', 'agg': 'count'},
    {'groupby': ['channel'], 'select': 'app', 'agg': 'count'}
    ]

# Apply all the groupby transformations
    for spec in GROUPBY_AGGREGATIONS:
    
    # Name of the aggregation we're applying
        agg_name = spec['agg_name'] if 'agg_name' in spec else spec['agg']
    
    # Info
        print("Grouping by {}, and aggregating {} with {}".format(
        spec['groupby'], spec['select'], agg_name
        ))
    
    # Unique list of features to select
        all_features = list(set(spec['groupby'] + [spec['select']]))
    
    # Name of new feature
        new_feature = '{}_{}_{}'.format('_'.join(spec['groupby']), agg_name, spec['select'])
    
    # Perform the groupby
        gp = df[all_features]. \
        groupby(spec['groupby'])[spec['select']]. \
        agg(spec['agg']). \
        reset_index(). \
        rename(index=str, columns={spec['select']: new_feature})
        
    # Merge back to X_train
        df=df.merge(gp, on=spec['groupby'], how='left')
    return df

In [37]:
newtraindata=groupByFeatures(newtraindata)

Grouping by ['ip', 'app', 'channel'], and aggregating dayofclick with var
Grouping by ['ip', 'app', 'os'], and aggregating hourofclick with var
Grouping by ['ip', 'dayofclick', 'channel'], and aggregating hourofclick with var
Grouping by ['ip', 'dayofclick', 'hourofclick'], and aggregating channel with count
Grouping by ['ip', 'app'], and aggregating channel with count
Grouping by ['ip', 'app', 'os'], and aggregating channel with count
Grouping by ['ip', 'app', 'dayofclick', 'hourofclick'], and aggregating channel with count
Grouping by ['ip', 'app', 'channel'], and aggregating hourofclick with mean
Grouping by ['app'], and aggregating ip with AvgViewPerDistinct
Grouping by ['app'], and aggregating channel with count
Grouping by ['channel'], and aggregating app with count


In [38]:
import gc
gc.collect()

465

In [39]:
newtraindata=newtraindata.drop('click_time',axis=1)



ValueError: labels ['click_time'] not contained in axis

In [41]:
newtraindata.columns

Index(['ip', 'app', 'device', 'os', 'channel', 'is_attributed', 'dayofclick',
       'hourofclick', 'minofclick', 'secofclick',
       'ip_app_channel_var_dayofclick', 'ip_app_os_var_hourofclick',
       'ip_dayofclick_channel_var_hourofclick',
       'ip_dayofclick_hourofclick_count_channel', 'ip_app_count_channel',
       'ip_app_os_count_channel',
       'ip_app_dayofclick_hourofclick_count_channel',
       'ip_app_channel_mean_hourofclick', 'app_AvgViewPerDistinct_ip',
       'app_count_channel', 'channel_count_app'],
      dtype='object')

In [42]:
import xgboost as xgb

# Split into X and y
y = newtraindata['is_attributed']
X = newtraindata.as_matrix(columns=['ip', 'app', 'device', 'os', 'channel',  'dayofclick',
       'hourofclick', 'minofclick', 'secofclick',
       'ip_app_channel_var_dayofclick', 'ip_app_os_var_hourofclick',
       'ip_dayofclick_channel_var_hourofclick',
       'ip_dayofclick_hourofclick_count_channel', 'ip_app_count_channel',
       'ip_app_os_count_channel',
       'ip_app_dayofclick_hourofclick_count_channel',
       'ip_app_channel_mean_hourofclick', 'app_AvgViewPerDistinct_ip',
       'app_count_channel', 'channel_count_app'])
# Create a model
# Params from: https://www.kaggle.com/aharless/swetha-s-xgboost-revised
clf_xgBoost = xgb.XGBClassifier(
    max_depth = 4,
    subsample = 0.8,
    colsample_bytree = 0.7,
    colsample_bylevel = 0.7,
    scale_pos_weight = 9,
    min_child_weight = 0,
    reg_alpha = 4,
    n_jobs = 4, 
    objective = 'binary:logistic'
)
# Fit the models
clf_xgBoost.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.7,
       colsample_bytree=0.7, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=0, missing=None, n_estimators=100,
       n_jobs=4, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=4, reg_lambda=1, scale_pos_weight=9, seed=None,
       silent=True, subsample=0.8)

In [43]:
del newtraindata

In [44]:
gc.collect()

94

In [45]:
del X

In [46]:
gc.collect()

0

In [47]:
del y

In [48]:
gc.collect()

0

In [49]:
testdata= pd.read_csv('test.csv',parse_dates=['click_time'],dtype=dtypes)

In [50]:
testdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18790469 entries, 0 to 18790468
Data columns (total 7 columns):
click_id      int64
ip            uint32
app           uint16
device        uint16
os            uint16
channel       uint16
click_time    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), uint16(4), uint32(1)
memory usage: 501.8 MB


In [51]:
import pickle

In [52]:
pickle.dump(clf_xgBoost, open('xgboost_4c_withmany.pkl', 'wb'))

In [53]:
testdata=timeFeature(testdata)
testdata.head()

,click_id,ip,app,device,os,channel,click_time,dayofclick,hourofclick,minofclick,secofclick
0,0,5744,9,1,3,107,2017-11-10 04:00:00,10,4,0,0
1,1,119901,9,1,3,466,2017-11-10 04:00:00,10,4,0,0
2,2,72287,21,1,19,128,2017-11-10 04:00:00,10,4,0,0
3,3,78477,15,1,13,111,2017-11-10 04:00:00,10,4,0,0
4,4,123080,12,1,13,328,2017-11-10 04:00:00,10,4,0,0


In [54]:
testdata=groupByFeatures(testdata)

Grouping by ['ip', 'app', 'channel'], and aggregating dayofclick with var
Grouping by ['ip', 'app', 'os'], and aggregating hourofclick with var
Grouping by ['ip', 'dayofclick', 'channel'], and aggregating hourofclick with var
Grouping by ['ip', 'dayofclick', 'hourofclick'], and aggregating channel with count
Grouping by ['ip', 'app'], and aggregating channel with count
Grouping by ['ip', 'app', 'os'], and aggregating channel with count
Grouping by ['ip', 'app', 'dayofclick', 'hourofclick'], and aggregating channel with count
Grouping by ['ip', 'app', 'channel'], and aggregating hourofclick with mean


MemoryError: 

In [40]:
testdata=testdata.drop('click_time',axis=1)


In [41]:
text_X = testdata.as_matrix(columns=['ip', 'app', 'device', 'os', 'channel',  'dayofclick',
       'hourofclick', 'minofclick', 'secofclick',
       'ip_app_channel_var_dayofclick', 'ip_app_os_var_hourofclick',
       'ip_dayofclick_channel_var_hourofclick',
       'ip_dayofclick_hourofclick_count_channel', 'ip_app_count_channel',
       'ip_app_os_count_channel',
       'ip_app_dayofclick_hourofclick_count_channel',
       'ip_app_channel_mean_hourofclick', 'app_AvgViewPerDistinct_ip',
       'app_count_channel', 'channel_count_app'])

In [42]:
pred=clf_xgBoost.predict_proba(text_X)

In [43]:
submission = pd.read_csv("sample_submission.csv")
submission['is_attributed'] = pred[:,1]
submission.to_csv("submission_4c_withmany.csv", index=False)